In [1]:
from utils import *
from dataset import *
from constants import *
from models.BaseModels import *
from models.DeepModels import *
from models.DeepUtils import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from dataset import Participant
from torch.utils.data import DataLoader

random.seed(RANDOM_STATE)

In [2]:
part_name = 's6'
use_saved = True

In [3]:
if use_saved and os.path.exists(f'saved/{part_name}.pkl'):
    participant = Participant.load_from_pickle(f'saved/{part_name}.pkl')
else:
    participant = Participant(part_name, data_path=DATA_PATH_NOTEBOOK, alpha=ALPHA)
    saved_dir = os.path.join(os.getcwd(), 'saved')
    if not os.path.exists(saved_dir):
        os.makedirs(saved_dir)
    with open(f'saved/{part_name}.pkl', 'wb') as f:
        pickle.dump(participant, f, pickle.HIGHEST_PROTOCOL)

In [4]:
test_size = 0.3
pca_expl_var = 0.95

## Execution

<span style='color: red'>WARNING</span>: computing all features takes around 7 minutes (Intel Core i7-7700K)

In [5]:
if use_saved and os.path.exists(f'saved/ex_features_{part_name}_mvt.h5'):
    ex_features = pd.read_hdf(f'saved/ex_features_{part_name}_mvt.h5', 'df')
else:
    ex_features = participant.get_features_all_sessions_mvt('E')
    ex_features.to_hdf(f'saved/ex_features_{part_name}_mvt.h5', 'df', mode='w', data_columns=True)

In [6]:
print(f'The dataset contains {ex_features.shape[0]} samples and {ex_features.shape[1]} features.')
print(f'The {len(participant.relevant_channels_ex)} relevant channels are located in the following locations:')
regions = [participant.channels_locations[i] for i in [channel.idx for channel in participant.relevant_channels_ex]]
channels_per_regions = {}
for region in set(regions):
    channels_per_regions[region] = regions.count(region)
print(channels_per_regions)

The dataset contains 128 samples and 4897 features.
The 68 relevant channels are located in the following locations:
{'paracentral': 3, 'WM_paracentral': 1, 'precentral': 26, 'insula': 3, 'WM_insula': 1, 'superiorfrontal': 5, 'caudalmiddlefrontal': 4, 'supramarginal': 3, 'postcentral': 16, 'WM_precentral': 6}


Let's create a baseline by taking the same number of channels, but without checking whether they are responsive:

<span style='color: red'>WARNING</span>: computing all features takes around 7 minutes (Intel Core i7-7700K)

In [7]:
use_saved = False
if use_saved and os.path.exists(f'saved/ex_baseline_features_{part_name}_mvt.h5'):
    ex_baseline_features = pd.read_hdf(f'saved/ex_baseline_features_{part_name}_mvt.h5', 'df')
else:
    ex_baseline_features = participant.get_features_all_sessions_rnd(len(participant.relevant_channels_ex), movtype='E')
    ex_baseline_features.to_hdf(f'saved/ex_baseline_features_{part_name}_mvt.h5', 'df', mode='w', data_columns=True)
use_saved = True

random_channels [<dataset.Channel object at 0x000001A5DA994E60>, <dataset.Channel object at 0x000001A5DA994F80>, <dataset.Channel object at 0x000001A5DA9950A0>, <dataset.Channel object at 0x000001A5DA9951C0>, <dataset.Channel object at 0x000001A5DA9952E0>, <dataset.Channel object at 0x000001A5DA995E20>, <dataset.Channel object at 0x000001A5DA996960>, <dataset.Channel object at 0x000001A5DA996BA0>, <dataset.Channel object at 0x000001A5DA997020>, <dataset.Channel object at 0x000001A5DA997260>, <dataset.Channel object at 0x000001A5DA9975C0>, <dataset.Channel object at 0x000001A5DA9976E0>, <dataset.Channel object at 0x000001A5DA997800>, <dataset.Channel object at 0x000001A5DA997C80>, <dataset.Channel object at 0x000001A5DA997DA0>, <dataset.Channel object at 0x000001A5DA9EC4A0>, <dataset.Channel object at 0x000001A5DA9ECB60>, <dataset.Channel object at 0x000001A5DA9ECC80>, <dataset.Channel object at 0x000001A5DA9ECDA0>, <dataset.Channel object at 0x000001A5DA9ECEC0>, <dataset.Channel object

C:\Users\sarah\AppData\Local\Temp\ipykernel_11280\1589942973.py:6: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  ex_baseline_features.to_hdf(f'saved/ex_baseline_features_{part_name}_mvt.h5', 'df', mode='w', data_columns=True)


In [8]:
print(f'The baseline dataset contains {ex_baseline_features.shape[0]} samples and {ex_baseline_features.shape[1]} features.')

The baseline dataset contains 128 samples and 2737 features.


### Train a model on the baseline features (Logistic Regression)

In [8]:
X = ex_baseline_features.drop('label', axis=1)
y = ex_baseline_features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=RANDOM_STATE)

logreg = LogisticRegressionModel()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.64


## Let's now do the analysis for the responsive channels

In [30]:
X = ex_features.drop('label', axis=1)
y = ex_features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=RANDOM_STATE)

### Train a model (Logistic Regression)

In [31]:
logreg = LogisticRegressionModel()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.77


#### With PCA

In [12]:
logreg = LogisticRegressionModel(use_pca=True, expl_var=0.95)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.69


### Train a model (SVM)

In [13]:
svm = SVMModel()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.62


#### With PCA

In [14]:
svm = SVMModel(use_pca=True, expl_var=0.95)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.56


### Train a model (Random Forest)

In [15]:
svm = RandomForestModel()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.54


### Train a model (Multi-Layer Perceptron)

In [16]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

mlp = MLP(X_train.shape[1], 2, layers=(16, 16))
trainset = DfDataset(X_train, y_train)
valset = DfDataset(X_val, y_val)
train_loader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=4)
val_loader = DataLoader(valset, batch_size=4, shuffle=False, num_workers=4)

trainer = Trainer(mlp, 0.1, 20, 4, save_path='saved/mlp.pth', device=device)
trainer.train(train_loader, val_loader)

Epoch 1/20 - Training: 100%|██████████| 23/23 [00:03<00:00,  6.56it/s]


Epoch: 1 	Training Loss: 9.571020 	Training Acc: 0.505618


Epoch 1/20 - Validation: 100%|██████████| 5/5 [00:02<00:00,  1.73it/s]


Epoch: 1 	Validation Loss: 11.290998 	Validation Acc: 0.421053
Validation loss decreased (inf --> 11.290998). Saving model ...


Epoch 2/20 - Training: 100%|██████████| 23/23 [00:03<00:00,  7.56it/s]


...

Epoch: 19 	Validation Loss: 0.645797 	Validation Acc: 0.631579


Epoch 20/20 - Training: 100%|██████████| 23/23 [00:02<00:00,  8.03it/s]


Epoch: 20 	Training Loss: 0.649542 	Training Acc: 0.584270


Epoch 20/20 - Validation: 100%|██████████| 5/5 [00:02<00:00,  1.85it/s]

Epoch: 20 	Validation Loss: 0.644686 	Validation Acc: 0.631579


In [28]:
testset = DfDataset(X_test, y_test)
acc = 0
for input, label in testset:
    pred = trainer.model(input)
    if torch.argmax(pred) == label:
        acc += 1

acc /= len(testset)
print(f"Accuracy: {acc:.2f}")

Accuracy: 0.55


## Observation

<span style='color: red'>WARNING</span>: computing all features takes around 7 minutes (Intel Core i7-7700K)

In [9]:
if use_saved and os.path.exists(f'saved/obs_features_{part_name}_mvt.h5'):
    obs_features = pd.read_hdf(f'saved/obs_features_{part_name}_mvt.h5', 'df')
else:
    obs_features = participant.get_features_all_sessions_mvt('O')
    obs_features.to_hdf(f'saved/obs_features_{part_name}_mvt.h5', 'df', mode='w', data_columns=True)

In [10]:
print(f'The dataset contains {obs_features.shape[0]} samples and {obs_features.shape[1]} features.')

The dataset contains 128 samples and 3673 features.


Let's create a baseline by taking the same number of channels, but without checking whether they are responsive:

In [11]:
use_saved = False
if use_saved and os.path.exists(f'saved/obs_baseline_features_{part_name}_mvt.h5'):
    obs_baseline_features = pd.read_hdf(f'saved/obs_baseline_features_{part_name}_mvt.h5', 'df')
else:
    obs_baseline_features = participant.get_features_all_sessions_rnd(len(participant.relevant_channels_obs), movtype='O')
    obs_baseline_features.to_hdf(f'saved/obs_baseline_features_{part_name}_mvt.h5', 'df', mode='w', data_columns=True)
use_saved = True

random_channels [<dataset.Channel object at 0x000001A5DA994E60>, <dataset.Channel object at 0x000001A5DA994F80>, <dataset.Channel object at 0x000001A5DA9950A0>, <dataset.Channel object at 0x000001A5DA9951C0>, <dataset.Channel object at 0x000001A5DA9952E0>, <dataset.Channel object at 0x000001A5DA995E20>, <dataset.Channel object at 0x000001A5DA996960>, <dataset.Channel object at 0x000001A5DA996BA0>, <dataset.Channel object at 0x000001A5DA997020>, <dataset.Channel object at 0x000001A5DA997260>, <dataset.Channel object at 0x000001A5DA9975C0>, <dataset.Channel object at 0x000001A5DA9976E0>, <dataset.Channel object at 0x000001A5DA997800>, <dataset.Channel object at 0x000001A5DA997C80>, <dataset.Channel object at 0x000001A5DA997DA0>, <dataset.Channel object at 0x000001A5DA9EC4A0>, <dataset.Channel object at 0x000001A5DA9ECB60>, <dataset.Channel object at 0x000001A5DA9ECC80>, <dataset.Channel object at 0x000001A5DA9ECDA0>, <dataset.Channel object at 0x000001A5DA9ECEC0>, <dataset.Channel object

C:\Users\sarah\AppData\Local\Temp\ipykernel_11280\209105401.py:6: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  obs_baseline_features.to_hdf(f'saved/obs_baseline_features_{part_name}_mvt.h5', 'df', mode='w', data_columns=True)


In [14]:
print(f'The baseline dataset contains {ex_baseline_features.shape[0]} samples and {ex_baseline_features.shape[1]} features.')

The baseline dataset contains 128 samples and 2737 features.


### Train a model on the baseline features (Logistic Regression)

In [15]:
X = obs_baseline_features.drop('label', axis=1)
y = obs_baseline_features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=RANDOM_STATE)

logreg = LogisticRegressionModel()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.49


## Let's now do the analysis for the responsive channels

In [16]:
X = obs_features.drop('label', axis=1)
y = obs_features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=RANDOM_STATE)

### Train a model (Logistic Regression)

In [17]:
logreg = LogisticRegressionModel()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.62


#### With PCA

In [33]:
logreg = LogisticRegressionModel(use_pca=True, expl_var=0.95)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.51


### Train a model (SVM)

In [34]:
svm = SVMModel()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.51


#### With PCA

In [35]:
svm = SVMModel(use_pca=True, expl_var=0.95)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.56


### Train a model (Random Forest)

In [36]:
svm = RandomForestModel()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.59
